# Idea

Predict returns from prices and volume traded at previous periods using various models.

# Imports

In [82]:
import pandas as pd

from ipynb.fs.full.utility_functions import (
    get_price_data,
    get_binance_px,
    get_train_test_data,
)
from datetime import datetime

# Get Historical Data For Cryptocurrencies

In [2]:
# The universe is based on this snapshot from December 20, 2020: https://coinmarketcap.com/historical/20201220/

univ = [
    "BTCUSDT", "ETHUSDT", "ADAUSDT", "BNBUSDT", "XRPUSDT", "DOTUSDT", "MATICUSDT", "LTCUSDT", "BCHUSDT",
    "LINKUSDT", "XLMUSDT", "USDCUSDT", "EOSUSDT", "TRXUSDT", "XTZUSDT", "FILUSDT", "NEOUSDT", "DAIUSDT",
    "DASHUSDT", "VETUSDT", "ATOMUSDT", "AAVEUSDT", "UNIUSDT", "GRTUSDT", "THETAUSDT", "IOTAUSDT", "BUSDUSDT",
    "ZECUSDT", "YFIUSDT", "ETCUSDT", "WAVESUSDT", "COMPUSDT", "SNXUSDT", "DOGEUSDT", "MKRUSDT", "ZILUSDT",
    "SUSHIUSDT", "KSMUSDT", "OMGUSDT", "ONTUSDT", "ALGOUSDT", "EGLDUSDT", "BATUSDT", "DGBUSDT", "ZRXUSDT",
    "TUSDUSDT", "QTUMUSDT", "ICXUSDT", "AVAXUSDT", "RENUSDT", "HBARUSDT", "NEARUSDT", "LRCUSDT", "CELOUSDT",
    "KNCUSDT", "LSKUSDT", "OCEANUSDT", "QNTUSDT", "USTUSDT", "BANDUSDT", "MANAUSDT", "ENJUSDT", "ANTUSDT",
    "BNTUSDT", "ZENUSDT", "NMRUSDT", "RVNUSDT", "IOSTUSDT", "OXTUSDT", "CRVUSDT", "MATICUSDT", "HNTUSDT",
    "BALUSDT", "CHZUSDT"
]

# px = get_price_data(univ, '4h', False, './class_project_input_prices.csv')
# px

In [3]:
# popular_univ = univ[["BTCUSDT", "ETHUSDT", "ADAUSDT", "BNBUSDT", "XRPUSDT"]]

In [69]:
from binance.client import Client as bnb_client

client = bnb_client(tld='US')
px = {}
freq = '4h'

univ = ["BTCUSDT"]

for coin in univ:
    print(f"Downloading data for symbol {coin}")
    data = get_binance_px(client, coin, freq)
    px[coin] = data.set_index('open_time')['close']

px = pd.DataFrame(px).astype(float)

In [70]:
px

,BTCUSDT
open_time,
2020-12-20 00:00:00,23353.97
2020-12-20 04:00:00,23604.24
2020-12-20 08:00:00,23549.50
2020-12-20 12:00:00,23880.85
2020-12-20 16:00:00,23932.71
...,...
2025-11-03 20:00:00,106350.88
2025-11-04 00:00:00,107178.80
2025-11-04 04:00:00,104620.99


predict close price for row (t, t+1) from:

- (t-1, t): open, high, low, close, volume, quote_volume, num_trades, taker_base_volume, taker_quote_volume, trade hour
- (t-2, t-1): open, high, low, close, volume, quote_volume, num_trades, taker_base_volume, taker_quote_volume, trade hour
- (t-3, t-2): ...
- (t-7, t-6): ... (for seasonality of a day)

In [98]:
def construct_look_back_data(input_data):
    input_data = input_data.drop(columns=["ignore"])
    input_data.set_index('open_time', inplace=True)
    input_data["period_return"] = input_data["close"].astype(float) / input_data["close"].astype(float).shift() - 1
    
    raw_input_data_back_1 = input_data.rename(columns=lambda x: x + "_back_1").shift()
    raw_input_data_back_2 = input_data.rename(columns=lambda x: x + "_back_2").shift(2)
    raw_input_data_back_3 = input_data.rename(columns=lambda x: x + "_back_3").shift(3)
    raw_input_data_back_6 = input_data.rename(columns=lambda x: x + "_back_6").shift(6)
    
    merged_input_data = pd.concat(
        [raw_input_data_back_1, raw_input_data_back_2, raw_input_data_back_3, raw_input_data_back_6], axis=1
    )
    
    original_input_columns = [
        "open", "high", "low", "close", "volume", "quote_volume", "num_trades", "taker_base_volume",
        "taker_quote_volume", "period_return",
    ]

    new_input_columns = [
        *map(lambda x: x + "_back_1", original_input_columns),
        *map(lambda x: x + "_back_2", original_input_columns),
        *map(lambda x: x + "_back_3", original_input_columns),
        *map(lambda x: x + "_back_6", original_input_columns),
    ]
    
    input_x = merged_input_data[new_input_columns]
    input_x["open_hour"] = input_x.index.hour
    input_x = input_x.astype(float)
    input_x = input_x.loc[~np.isnan(input_x).any(axis=1)]
    
    y = input_data["period_return"].loc[input_x.index]
    
    return input_x, y

In [99]:
all_x, all_y = construct_look_back_data(data)

t = datetime(2024, 1, 1, 0, 0)

train_x, test_x = get_train_test_data(all_x, t)
train_y, test_y = get_train_test_data(all_y, t)

/var/folders/46/nmm5hcbx71d6nqy2066wsyjc0000gn/T/ipykernel_1103/3347170795.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_x["open_hour"] = input_x.index.hour


In [104]:
import statsmodels.api as sm
import numpy as np

train_x = sm.add_constant(train_x)
model = sm.OLS(train_y, train_x)
model

In [105]:
results = model.fit()
results.params

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


array([ 1.25841784e-03,  1.15977392e-05,  9.43677261e-08,  5.45928074e-07,
       -2.11223374e-06,  2.18543346e-05, -4.66804768e-10,  1.82859575e-08,
       -4.25148376e-05,  9.03208518e-10,  2.87883012e-02,  3.91491659e-06,
        5.99597433e-08,  3.62001409e-07, -9.17435782e-06,  1.46191884e-05,
        2.84934792e-10, -6.50824971e-08, -3.10942517e-05, -5.35933377e-10,
       -2.42456021e-02,  2.18282727e-06,  2.36892400e-07, -5.32003600e-07,
       -7.56528703e-06,  2.52856919e-06, -4.12656123e-10, -5.46119310e-08,
       -1.22385899e-05,  1.36340844e-09,  1.25397665e-01, -2.79791451e-07,
        7.46240240e-08, -1.31992785e-06,  1.86292931e-06, -6.72810661e-05,
        2.20106582e-09,  7.40227234e-11,  1.40579165e-04, -4.83529041e-09,
       -8.33848699e-02,  4.71061418e-05])

In [106]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     2.321
Date:                Tue, 04 Nov 2025   Prob (F-statistic):           3.82e-06
Time:                        08:09:10   Log-Likelihood:                 18876.
No. Observations:                6633   AIC:                        -3.767e+04
Df Residuals:                    6591   BIC:                        -3.738e+04
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0013      0.001      1.462      0.144      -0.000       0.003
x1           1.16e-05   8.15e-06      1.423      0.155   -4.37e-06    2.76e-05
x2          9.437e-08   1.29e-07      0.734      0.463   -1.58e-07    3.46e-07
x3          5.459e-07   6.68e-07      0.817      0.414   -7.63e-07    1.86e-06
x4         -2.112e-06   1.19e-06     -1.779      0.075   -4.44e-06    2.15e-07
x5          2.185e-05    3.7e-05      0.591      0.554   -5.06e-05    9.43e-05
x6         -4.668e-10   1.08e-09     -0.431      0.667   -2.59e-09    1.66e-09
x7          1.829e-08   1.28e-07      0.143      0.887   -2.33e-07    2.69e-07
x8         -4.251e-05   7.47e-05     -0.569      0.569      -0.000       0.000
x9          9.032e-10   2.16e-09      0.419      0.676   -3.33e-09    5.13e-09
x10            0.0288      0.043      0.664      0.507      -0.056       0.114
x11         3.915e-06   8.11e-06      0.483      0.629    -1.2e-05    1.98e-05
x12         5.996e-08   1.29e-07      0.466      0.641   -1.92e-07    3.12e-07
x13          3.62e-07   6.65e-07      0.544      0.586   -9.41e-07    1.67e-06
x14        -9.174e-06   8.38e-06     -1.095      0.273   -2.56e-05    7.25e-06
x15         1.462e-05   3.71e-05      0.394      0.693    -5.8e-05    8.73e-05
x16         2.849e-10   1.09e-09      0.261      0.794   -1.85e-09    2.42e-09
x17        -6.508e-08   1.42e-07     -0.458      0.647   -3.44e-07    2.13e-07
x18        -3.109e-05   7.49e-05     -0.415      0.678      -0.000       0.000
x19        -5.359e-10   2.16e-09     -0.248      0.804   -4.78e-09    3.71e-09
x20           -0.0242      0.043     -0.558      0.577      -0.109       0.061
x21         2.183e-06   1.16e-06      1.889      0.059   -8.19e-08    4.45e-06
x22         2.369e-07   1.29e-07      1.843      0.065   -1.51e-08    4.89e-07
x23         -5.32e-07   6.51e-07     -0.818      0.413   -1.81e-06    7.43e-07
x24        -7.565e-06   8.32e-06     -0.909      0.363   -2.39e-05    8.75e-06
x25         2.529e-06   3.69e-05      0.068      0.945   -6.98e-05    7.49e-05
x26        -4.127e-10   1.08e-09     -0.381      0.703   -2.54e-09    1.71e-09
x27        -5.461e-08   1.29e-07     -0.424      0.672   -3.07e-07    1.98e-07
x28        -1.224e-05   7.43e-05     -0.165      0.869      -0.000       0.000
x29         1.363e-09   2.15e-09      0.634      0.526   -2.85e-09    5.58e-09
x30            0.1254      0.043      2.913      0.004       0.041       0.210
x31        -2.798e-07   1.12e-06     -0.250      0.803   -2.47e-06    1.91e-06
x32         7.462e-08   1.29e-07      0.579      0.563   -1.78e-07    3.27e-07
x33         -1.32e-06   6.39e-07     -2.064      0.039   -2.57e-06   -6.65e-08
x34         1.863e-06   1.19e-06      1.567      0.117   -4.67e-07    4.19e-06
x35        -6.728e-05   3.68e-05     -1.828      0.068      -0.000    4.88e-06
x3

In [107]:
test_x = sm.add_constant(test_x)

predicted_y = model.predict(test_x)
predicted_y

ValueError: shapes (6633,42) and (4042,42) not aligned: 42 (dim 1) != 4042 (dim 0)

In [88]:
train_x.shape

(6633, 42)

In [90]:
test_x.shape

(4042, 41)

In [91]:
train_x.columns

Index(['const', 'open_back_1', 'high_back_1', 'low_back_1', 'close_back_1',
       'volume_back_1', 'quote_volume_back_1', 'num_trades_back_1',
       'taker_base_volume_back_1', 'taker_quote_volume_back_1',
       'period_return_back_1', 'open_back_2', 'high_back_2', 'low_back_2',
       'close_back_2', 'volume_back_2', 'quote_volume_back_2',
       'num_trades_back_2', 'taker_base_volume_back_2',
       'taker_quote_volume_back_2', 'period_return_back_2', 'open_back_3',
       'high_back_3', 'low_back_3', 'close_back_3', 'volume_back_3',
       'quote_volume_back_3', 'num_trades_back_3', 'taker_base_volume_back_3',
       'taker_quote_volume_back_3', 'period_return_back_3', 'open_back_6',
       'high_back_6', 'low_back_6', 'close_back_6', 'volume_back_6',
       'quote_volume_back_6', 'num_trades_back_6', 'taker_base_volume_back_6',
       'taker_quote_volume_back_6', 'period_return_back_6', 'open_hour'],
      dtype='object')

In [92]:
test_x.columns

Index(['open_back_1', 'high_back_1', 'low_back_1', 'close_back_1',
       'volume_back_1', 'quote_volume_back_1', 'num_trades_back_1',
       'taker_base_volume_back_1', 'taker_quote_volume_back_1',
       'period_return_back_1', 'open_back_2', 'high_back_2', 'low_back_2',
       'close_back_2', 'volume_back_2', 'quote_volume_back_2',
       'num_trades_back_2', 'taker_base_volume_back_2',
       'taker_quote_volume_back_2', 'period_return_back_2', 'open_back_3',
       'high_back_3', 'low_back_3', 'close_back_3', 'volume_back_3',
       'quote_volume_back_3', 'num_trades_back_3', 'taker_base_volume_back_3',
       'taker_quote_volume_back_3', 'period_return_back_3', 'open_back_6',
       'high_back_6', 'low_back_6', 'close_back_6', 'volume_back_6',
       'quote_volume_back_6', 'num_trades_back_6', 'taker_base_volume_back_6',
       'taker_quote_volume_back_6', 'period_return_back_6', 'open_hour'],
      dtype='object')

In [96]:
set(train_x.columns).difference(set(test_x.columns))

{'const'}